In [10]:
import ipeadatapy as ipea
import pandas as pd
import numpy as np
import json

In [4]:
series = ipea.list_series()
series.tail()

,CODE,NAME
9603,ADH_RDPC4_URB,Renda domiciliar per capita média do 4º quint...
9604,ADH_RDPC5_RUR,Renda domiciliar per capita média do quinto ma...
9605,ADH_RDPC5_URB,Renda domiciliar per capita média do quinto ma...
9606,ADH_THEIL_RUR,Índice de Theil - L - áreas rurais (Atlas DH -...
9607,ADH_THEIL_URB,Índice de Theil - L - áreas urbanas (Atlas DH ...


In [45]:
series.loc[series.NAME == "Salário mínimo real"]

,CODE,NAME
7324,GAC12_SALMINRE12,Salário mínimo real


In [48]:
ipea.describe("GAC12_SALMINRE12")

,Salário mínimo real
Name,Salário mínimo real
Code,GAC12_SALMINRE12
Big Theme,Macroeconômico
Theme code,13
Source,Instituto de Pesquisa Econômica Aplicada
Source acronym,IPEA
Comment,O Salário mínimo nominal é o menor salário def...
Last update,2023-09-12T11:36:00.637-03:00
Frequency,Mensal
Measure,R$ (do último mês)


In [108]:
bases = {
    "exportacao": ipea.describe("BPN_XTV"),
    "importacao": ipea.describe("BPN_MTV"),
    "base_monetaria": ipea.describe("BM_M0FN"),
    "inflacao": ipea.describe("IGP_IGPDIG"),
    "cambio": ipea.describe("BM_ERCF"),
    "divida_externa": ipea.describe("BM_DEXR"),
    "pib_industria": ipea.describe("SCN10_VAINDN10"),
    "pib": ipea.describe("SCN10_PIBN10"),
    "salario": ipea.describe("GAC12_SALMINRE12")
}

pd.concat([base for base in bases.values()], axis = 1).to_excel("data/descricao_bases.xlsx")
pd.read_excel("data/descricao_bases.xlsx", index_col = 0).T.reset_index(drop = True)

,Name,Code,Big Theme,Theme code,Source,Source acronym,Comment,Last update,Frequency,Measure,Unit,Status
0,Exportações - (FOB) (Antiga metod. - BPM5) - I...,BPN_XTV,Macroeconômico,5,"Banco Central do Brasil, Balanço de Pagamentos...",Bacen/BP (BPM5),"Até dezembro de 1958, dados da revista Conjunt...",2015-01-23T18:16:50-02:00,Anual,US$,milhões,I
1,Importações - (FOB) (Antiga metod. - BPM5) - I...,BPN_MTV,Macroeconômico,5,"Banco Central do Brasil, Balanço de Pagamentos...",Bacen/BP (BPM5),"Até dezembro de 1958, dados da revista Conjunt...",2015-01-23T18:16:49-02:00,Anual,US$,milhões,I
2,Base monetária restrita -M0 - fim de período,BM_M0FN,Macroeconômico,3,"Banco Central do Brasil, Notas Econômico-finan...",Bacen/Notas Imprensa/Moeda,A base monetária restrita é a soma do papel-mo...,2023-01-27T18:50:00-03:00,Anual,R$,milhões,A
3,IGP-DI,IGP_IGPDIG,Macroeconômico,9,"Fundação Getulio Vargas, Conjuntura Econômica ...",FGV/Conj. Econ. - IGP,O Índice Geral de Preços (IGP) registra o ritm...,2023-01-06T15:12:01.24-03:00,Anual,(% a.a.),NaN,A
4,Taxa de câmbio - R$ / US$ - comercial - compra...,BM_ERCF,Macroeconômico,7,"Banco Central do Brasil, Boletim, Seção Balanç...",Bacen/Boletim/BP,Taxa de câmbio é o preço de uma moeda estrange...,2023-01-02T14:40:00-03:00,Anual,R$,NaN,A
5,Dívida externa - registrada - INATIVA,BM_DEXR,Macroeconômico,10,"Banco Central do Brasil, Boletim, Seção Balanç...",Bacen/Boletim/BP,"Para 1889-1945: Abreu, Marcelo de Paiva (Org.)...",2009-06-26T16:30:13-03:00,Anual,US$,milhões,I
6,PIB - indústria,SCN10_VAINDN10,Macroeconômico,8,Instituto Brasileiro de Geografia e Estatístic...,IBGE/SCN Anual,O produto interno bruto (PIB) é o total dos be...,2023-03-02T17:16:01-03:00,Anual,R$,milhões,A
7,PIB - preços de mercado,SCN10_PIBN10,Macroeconômico,8,Instituto Brasileiro de Geografia e Estatístic...,IBGE/SCN Anual,O produto interno bruto (PIB) é o total dos be...,2023-03-02T17:16:01-03:00,Anual,R$,milhões,A
8,Salário mínimo real,GAC12_SALMINRE12,Macroeconômico,13,Instituto de Pesquisa Econômica Aplicada,IPEA,O Salário mínimo nominal é o menor salário def...,2023-09-12T11:36:00.637-03:00,Mensal,R$ (do último mês),,A


In [103]:
for base, dados in bases.items():
    bases[base] = list(dados.to_dict().values())[0]
    bases[base]["dataframe"] = (
        ipea.timeseries(bases[base]["Code"]).reset_index()
        .rename({f"VALUE ({bases[base]['Measure']})": "VALUE"}, axis = 1)
        .assign(
            nome = base,
            unidade = bases[base]['Unit'], 
            medida = bases[base]['Measure']
        ).groupby("YEAR").agg("first").reset_index()
    )

df = (
    pd.concat([base["dataframe"] for base in bases.values()])
    .merge(
        pd.read_csv("data/deflator.csv")
        .rename({"DPCERG3A086NBEA": "deflator"}, axis = 1)
        .assign(DATE = lambda _: pd.to_datetime(_.DATE),
                deflator = lambda _: _.deflator / 100), 
        how = "left"
    )
    .pipe(lambda DF: DF.merge(DF
        .query("nome == 'cambio'")
        .rename({"VALUE": "cambio"}, axis = 1)
        .filter(["DATE", "cambio"]), 
    how = "left")
    )
)

bases_inalteradas = ["cambio", "base_monetaria", "salario"]

relatorio = (df
 .query("DATE.dt.year > 1940 and DATE.dt.year < 2000")
 .assign(
    unidade = lambda _: 
        np.where(_.unidade == 'milhões', 1000000,
        np.where(_.unidade.isna(), 1, np.nan)
    ),
    multiplicador = lambda _: np.where(np.isin(_.nome, bases_inalteradas), 1, 
        (_.unidade) / (
            (_.deflator ** ((_.medida == "US$") | (_.medida == "R$"))) * #Deflacionar os dolares
            (_.cambio ** (_.medida == "R$")) #Trocar reais por dolares
        )
    ),
    VALUE = lambda _: _.VALUE * _.multiplicador
).pivot_table(
    index = "YEAR",
    columns = "nome",
    values = "VALUE"
).reset_index()
 .rename_axis("", axis = 1)
)

In [105]:
df.to_csv("data/data-raw.csv")
df.head()

,YEAR,DATE,CODE,RAW DATE,DAY,MONTH,VALUE,nome,unidade,medida,deflator,cambio
0,1889,1889-01-01,BPN_XTV,1889-01-01T00:00:00-02:00,1,1,138.8,exportacao,milhões,US$,NaN,NaN
1,1890,1890-01-01,BPN_XTV,1890-01-01T00:00:00-02:00,1,1,128.2,exportacao,milhões,US$,NaN,NaN
2,1891,1891-01-01,BPN_XTV,1891-01-01T00:00:00-02:00,1,1,131.9,exportacao,milhões,US$,NaN,NaN
3,1892,1892-01-01,BPN_XTV,1892-01-01T00:00:00-02:00,1,1,150.0,exportacao,milhões,US$,NaN,NaN
4,1893,1893-01-01,BPN_XTV,1893-01-01T00:00:00-02:00,1,1,155.6,exportacao,milhões,US$,NaN,NaN


In [107]:
relatorio.to_csv("data/data.csv")
relatorio.to_excel("data/data.xlsx")
relatorio.head()

,YEAR,base_monetaria,cambio,divida_externa,exportacao,importacao,inflacao,pib,pib_industria,salario
0,1941,NaN,NaN,1.229368e+10,4.717269e+09,3.223383e+09,NaN,NaN,NaN,1055.100972
1,1942,NaN,7.061818e-15,1.058211e+10,4.539071e+09,2.372593e+09,NaN,3.746830e+10,NaN,982.305958
2,1943,NaN,7.061818e-15,9.455507e+09,4.838208e+09,2.856254e+09,NaN,4.341173e+10,NaN,825.841463
3,1944,NaN,7.000000e-15,7.384268e+09,5.634563e+09,3.533739e+09,NaN,5.376085e+10,NaN,1011.477493
4,1945,NaN,7.000000e-15,6.583365e+09,6.180687e+09,3.672199e+09,11.111055,6.130825e+10,NaN,800.753049
